In [62]:
import tensorflow as tf
from transformers import BertTokenizer, BertConfig
from transformers import BertModel
import nltk
import numpy as np
import torch
import re

In [63]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [74]:
def is_available_in_pos_list(dict_list, begin_pos, end_pos):
    for entry in dict_list:
        if((entry["start"]<=begin_pos and entry["end"]>begin_pos) or ((entry["start"]<end_pos and entry["end"]>=end_pos))):
            return True
    
    return False
'''
def get_bert_token_positions(input_text,token_list,start_from_pos=0,):
    
    pos_list = []                    
    
    name_to_match = input_text.lower().replace(" ","")
    remaining_name = input_text.lower().replace(" ","")
    
    name = ""
    count = start_from_pos

    for entry in token_list[start_from_pos:]:
        if(remaining_name.startswith(entry.strip("##").lower())):
            pos_list.append(count)
            remaining_name = remaining_name[len(entry.strip("##").lower()):]
            name = name + entry.strip("##").lower()
            if(name == name_to_match):
                break
        else:
            pos_list = []
            name = ""
            remaining_name = name_to_match
            if(remaining_name.startswith(entry.strip("##").lower())):                                 
                pos_list.append(count)                                                                
                remaining_name = remaining_name[len(entry.strip("##").lower()):]
                name = name + entry.strip("##").lower()    
                if(name == name_to_match):                                   
                    break

        count = count + 1

    return pos_list
'''
def get_bert_token_positions(input_text,token_list,start_from_pos=0,prior_partial_word=""):
    partial_word = ""

    pos_list = []                    
    
    if(prior_partial_word!=""):
        input_text = prior_partial_word + input_text 

    name_to_match = input_text.lower().replace(" ","")
    remaining_name = input_text.lower().replace(" ","")
    
    name = ""
    count = start_from_pos

    for entry in token_list[start_from_pos:]:
        entry_text = entry.strip("##").lower()
        if(entry_text.startswith(remaining_name) and (entry_text != remaining_name)):
            partial_word = remaining_name
            pos_list.append(count)
            break
             
        if(remaining_name.startswith(entry_text)):
            pos_list.append(count)
            remaining_name = remaining_name[len(entry_text):]
            name = name + entry_text
            if(name == name_to_match):
                break
        else:
            pos_list = []
            name = ""
            remaining_name = name_to_match
            if(remaining_name.startswith(entry.strip("##").lower())):                                 
                pos_list.append(count)                                                                
                remaining_name = remaining_name[len(entry.strip("##").lower()):]
                name = name + entry.strip("##").lower()    
                if(name == name_to_match):                                   
                    break

        count = count + 1
    
    return [pos_list,partial_word]

def process_string(string_input, entity_list):
    string_input = re.sub(r'Admission Date :\n([0-9/ ]*)\n', 'Admission Date : \g<1>\n', string_input)
    string_input = re.sub(r'Discharge Date :\n([0-9/ ]*)\n', 'Discharge Date : \g<1>\n', string_input)
    
    
    tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
    init_sentences = tokenizer.tokenize(string_input)
    
    sentences = []
    
    for entry in init_sentences:
        sentences.extend(entry.split("\n"))
    
    #config = BertConfig.from_pretrained('C:/Users/itsma/Documents/BERT_models/NCBI_BERT_pubmed_mimic_uncased_L-12_H-768_A-12')
    #config.output_hidden_states = True

    bert_tokenizer = BertTokenizer.from_pretrained('C:/Users/itsma/Documents/BERT_models/NCBI_BERT_pubmed_mimic_uncased_L-12_H-768_A-12')
    #bert_model = BertModel.from_pretrained("C:/Users/itsma/Documents/BERT_models/NCBI_BERT_pubmed_mimic_uncased_L-12_H-768_A-12",config=config)
    
    positions_covered = 0
    sentence_list = []
    
    for index in range(len(sentences)):
        new_dict_sentence = {}
        sentence = sentences[index]
        new_dict_sentence["sentence"] = sentence
        #new_dict_sentence["padding_length"] = padding_length
        #start_index_bert = max(0,index-padding_length)
        #end_index_bert = min(len(sentences),index+padding_length)

        bert_input = sentences[index]
        
        encodings = bert_tokenizer.encode(bert_input,add_special_tokens = True)
        new_dict_sentence["encodings"] = encodings
        input_ids = torch.tensor(encodings).unsqueeze(0)  
        #outputs = bert_model(input_ids)
        #bert_vector = outputs[2]
        bert_tokens = bert_tokenizer.convert_ids_to_tokens(encodings) #bert_tokenizer.tokenize(bert_input,add_special_tokens = True)
        new_dict_sentence["tokens"] = bert_tokens
        
        start_pos = 0
        #prior_pos = get_bert_token_positions(' '.join(sentences[start_index_bert:index]),bert_tokens)
        
        #if(len(prior_pos)>0):
            #start_pos = max(prior_pos)
            
        tokens = nltk.word_tokenize(sentence)
        pos_tokens = nltk.pos_tag(tokens)

        sentence_covered = ''
        prior_partial_word = ''
        label_list = [0] * len(encodings)
        
        for token in pos_tokens:
            new_dict = {}
            current_word = token[0]
            
            [bert_token_positions, partial_word] = get_bert_token_positions(current_word,bert_tokens,start_pos,prior_partial_word)
            
            #ec_list_layers = []
            
            if(len(bert_token_positions)==0):
                prior_partial_word = ""
                continue
            if(partial_word != ""):
                prior_partial_word = partial_word
                start_pos = bert_token_positions[-1]
            else:
                prior_partial_word = ""
                start_pos = bert_token_positions[-1] + 1
            token_position = string_input.find(current_word, positions_covered)
            spaces_between = string_input[positions_covered:token_position] 
            sentence_covered = sentence_covered + spaces_between + current_word
            positions_covered = token_position + len(current_word)
            begin_pos = token_position
            end_pos = positions_covered
            #bert_token_positions = get_bert_token_positions(current_word,bert_tokens,start_pos)[0]
           
            '''
            vec_list_layers = []
            
            if(len(bert_token_positions)==0):
                continue
            start_pos = bert_token_positions[-1] + 1
            '''
            entity_index=1
            for entity in entity_list:
                if(is_available_in_pos_list(entity,begin_pos,end_pos)):
                    for entry in bert_token_positions:
                        label_list[entry] = entity_index
                
                entity_index = entity_index + 1
                
                
        new_dict_sentence["labels"] = label_list

        sentence_list.append(new_dict_sentence)
        
    return sentence_list

In [75]:
class DischargeNote():
    def __init__(self, root,padding_length=0):
        self.xml_root = root
        self.padding_length = padding_length
        
    def process_note(self):
        root = self.xml_root
        text_section = root.find('TEXT')
        text = text_section.text
        
        tag_section = root.find('TAGS')
        event_list = []
        timex_list = []
        tlink_list = []
        sectime_list = []
        for child in tag_section:
            if(child.tag=='EVENT'):
                event_list.append(child.attrib)
            elif(child.tag=='TIMEX3'):
                timex_list.append(child.attrib)
            elif(child.tag=='TLINK'):
                tlink_list.append(child.attrib)
            elif(child.tag=='SECTIME'):
                sectime_list.append(child.attrib)
        
        event_list_new = []
        
        for sub in event_list:
            new_dict = {}
            new_dict["start"] = int(sub["start"])
            new_dict["end"] = int(sub["end"])
            event_list_new.append(new_dict)
        
        event_list_new = sorted(event_list_new, key = lambda i: i['start'])
        
        timex_list_new = []
        
        for sub in timex_list:
            new_dict = {}
            new_dict["start"] = int(sub["start"])
            new_dict["end"] = int(sub["end"])
            timex_list_new.append(new_dict)
        
        timex_list_new = sorted(timex_list_new, key = lambda i: i['start'])
        
        entity_list = []
        entity_list.append(event_list_new)
        entity_list.append(timex_list_new)
        
        
        self.processed_text = process_string(text,entity_list)
        
        sentences_length = len(self.processed_text)
        encoding_list = []
        label_list = []
        token_list = []
        
        for sentence_index in range(sentences_length):
            padding_unsuccess = True
            padding_length = self.padding_length
            
            encodings = []
            labels = []
            tokens = []
            while(padding_unsuccess):
                encodings = []
                labels = []
                tokens=[]
                
                begin_index = max(0,sentence_index-padding_length)
                end_index = min(sentences_length,sentence_index+padding_length)
                current_index = 1
                last_index = end_index - begin_index + 1
        
                if(begin_index==end_index):
                    entry = self.processed_text[begin_index]
                    encodings.extend(entry['encodings'])
                    tokens.extend(entry['tokens'])
                    labels.extend(entry['labels'])
                else:
                    for entry in self.processed_text[begin_index:end_index+1]:
                        if(current_index==1):
                            encodings.extend(entry['encodings'][:-1])
                            tokens.extend(entry['tokens'][:-1])
                            labels.extend(entry['labels'][:-1])
                        elif(current_index==last_index):
                            encodings.extend(entry['encodings'][1:])
                            tokens.extend(entry['tokens'][1:])
                            labels.extend(entry['labels'][1:])
                        else:
                            encodings.extend(entry['encodings'][1:-1])
                            tokens.extend(entry['tokens'][1:-1])
                            labels.extend(entry['labels'][1:-1])
                        current_index = current_index + 1

                if(len(encodings)<=512):
                    padding_unsuccess = False
                else:
                    padding_length = max(0,padding_length - 1)
            
            encoding_list.append(encodings)
            token_list.append(tokens)
            label_list.append(labels)
            
        return [encoding_list,token_list,label_list]

In [76]:
import os
import xml.etree.ElementTree as ET

In [77]:
file = "1.xml"

file_name = os.path.join("C:/Users/itsma/Documents/Capstone project/DS5500-capstone/train_data/", file)
tree = ET.parse(file_name)
root = tree.getroot()
discharge_note = DischargeNote(root,1)
results = discharge_note.process_note()

In [78]:
print(results[0][1])

[101, 9634, 3058, 1024, 5641, 1013, 2756, 1013, 2857, 11889, 3058, 1024, 2184, 1013, 5840, 1013, 2857, 102]


In [79]:
print(results[1][1])

['[CLS]', 'admission', 'date', ':', '09', '/', '29', '/', '1993', 'discharge', 'date', ':', '10', '/', '04', '/', '1993', '[SEP]']


In [80]:
print(results[2][1])

[0, 1, 0, 0, 2, 2, 2, 2, 2, 1, 0, 0, 2, 2, 2, 2, 2, 0]


In [76]:
# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []

# For every sentence...
for sent in sentences:
    # `encode` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    encoded_sent = tokenizer.encode(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'

                        # This function also supports truncation and conversion
                        # to pytorch tensors, but we need to do padding, so we
                        # can't use these features :( .
                        #max_length = 128,          # Truncate all sentences.
                        #return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.
    input_ids.append(encoded_sent)

# Print sentence 0, now as a list of IDs.
print('Original: ', sentences[0])
print('Token IDs:', input_ids[0])

NameError: name 'sentences' is not defined

In [81]:
from progressbar import ProgressBar
pbar = ProgressBar()

In [82]:
input_ids = []
labels = []

for file in pbar(os.listdir("C:/Users/itsma/Documents/Capstone project/DS5500-capstone/train_data/")):
    if file.endswith(".xml"):
        file_name = os.path.join("C:/Users/itsma/Documents/Capstone project/DS5500-capstone/train_data/", file)
        tree = ET.parse(file_name)
        root = tree.getroot()
        discharge_note = DischargeNote(root,1)
        results = discharge_note.process_note()
        
        sample_length = len(results[0])
        
        for index in range(sample_length):
            input_ids.append(results[0][index])
            labels.append(results[2][index])

100% |########################################################################|


In [83]:
import pickle

In [84]:
pickle.dump(input_ids,open("C:/Users/itsma/Documents/input_ids.pkl","wb"))
pickle.dump(labels,open("C:/Users/itsma/Documents/label.pkl","wb"))

In [42]:
config = BertConfig.from_pretrained('C:/Users/itsma/Documents/BERT_models/NCBI_BERT_pubmed_mimic_uncased_L-12_H-768_A-12')
config.output_hidden_states = True

bert_tokenizer = BertTokenizer.from_pretrained('C:/Users/itsma/Documents/BERT_models/NCBI_BERT_pubmed_mimic_uncased_L-12_H-768_A-12')
bert_model = BertModel.from_pretrained("C:/Users/itsma/Documents/BERT_models/NCBI_BERT_pubmed_mimic_uncased_L-12_H-768_A-12",config=config)

In [43]:
bert_input1 = "I love my Dog.He is Cute." 
bert_input2 = "He is Cute."
encodings1 = bert_tokenizer.encode(bert_input1,add_special_tokens = True)
input_ids1 = torch.tensor(encodings1).unsqueeze(0)  
encodings2 = bert_tokenizer.encode(bert_input2,add_special_tokens = True)
input_ids2 = torch.tensor(encodings2).unsqueeze(0) 

In [44]:
input_ids1

tensor([[  101,  1045,  2293,  2026,  3899,  1012,  2002,  2003, 10140,  1012,
           102]])

In [48]:
encodings1

[101, 1045, 2293, 2026, 3899, 1012, 2002, 2003, 10140, 1012, 102]

In [45]:
input_ids2

tensor([[  101,  2002,  2003, 10140,  1012,   102]])

In [50]:

[0] * 4 

[0, 0, 0, 0, 0, 0, 0, 0]

In [131]:
sample_list = [1,2,3,4,5,6]

In [135]:
sample_list[:-1]

[1, 2, 3, 4, 5]